In [51]:
import uuid
import httpx

payload = {
  "jsonrpc": "2.0",
  "id": 1,
  "method": "tools/call",
  "params": {
    "name": "roll_dice",
    "arguments": {
        "n_dice": 5
    }
  }
}

headers = {"Accept": "application/json, text/event-stream"}

r = httpx.post("http://localhost:8000/mcp", json=payload, headers=headers)
print(r.text)

event: message
data: {"jsonrpc":"2.0","id":1,"result":{"content":[{"type":"text","text":"[6,2,6,6,1]"}],"structuredContent":{"result":[6,2,6,6,1]},"isError":false}}




In [52]:
import uuid
import httpx

payload = {
  "jsonrpc": "2.0",
  "id": 1,
  "method": "tools/call",
  "params": {
    "name": "sum_dice",
    "arguments": {
        "rolls": [3, 6, 1, 3, 6, 1, 3, 4, 6, 3, 2, 3, 2, 2, 5, 2, 2, 2, 4, 6]
    }
  }
}

headers = {"Accept": "application/json, text/event-stream"}

r = httpx.post("http://localhost:8000/mcp", json=payload, headers=headers)
print(r.text)

event: message
data: {"jsonrpc":"2.0","id":1,"result":{"content":[{"type":"text","text":"66"}],"structuredContent":{"result":66},"isError":false}}




In [55]:
import ast
import json
from typing import Any, Dict, Iterable, Optional, Tuple

import httpx

MCP_URL = "http://localhost:8000/mcp"  # or ".../mcp/" if your server expects the trailing slash
HEADERS_SSE = {"Accept": "application/json, text/event-stream"}
HEADERS_JSON = {"Accept": "application/json"}

def _iter_sse_data_lines(resp: httpx.Response) -> Iterable[str]:
    """
    Yield JSON strings from an SSE response by reading 'data: ...' lines.
    """
    for line in resp.iter_lines():
        if not line:
            continue
        # httpx returns bytes; decode if needed
        if isinstance(line, bytes):
            line = line.decode("utf-8", errors="ignore")
        if line.startswith("data: "):
            yield line[len("data: "):].strip()

def _extract_result(obj: Dict[str, Any]) -> Tuple[Optional[Any], Optional[str]]:
    """
    Extract result or error message from an MCP JSON-RPC envelope.
    Returns (result_obj, error_message).
    """
    if "error" in obj and obj["error"]:
        msg = obj["error"].get("message") or str(obj["error"])
        return None, msg

    res = obj.get("result", {})
    # Prefer structuredContent.result if available
    sc = res.get("structuredContent", {})
    if isinstance(sc, dict) and "result" in sc:
        return sc["result"], None

    # Fallback to textual content
    content = res.get("content")
    if isinstance(content, list) and content and isinstance(content[0], dict) and content[0].get("type") == "text":
        return content[0].get("text"), None

    # Or raw result object
    if "result" in res:
        return res["result"], None

    return None, "No result in response"

def call_tool(
    name: str,
    arguments: Dict[str, Any],
    *,
    stream_preferred: bool = True,
    timeout: float = 15.0,
) -> Any:
    """
    Call an MCP tool and return the parsed result.
    Tries SSE first (if stream_preferred=True), then falls back to plain JSON.
    """
    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "tools/call",
        "params": {"name": name, "arguments": arguments},
    }

    with httpx.Client(timeout=timeout) as client:
        # Try SSE first
        if stream_preferred:
            try:
                with client.stream("POST", MCP_URL, json=payload, headers=HEADERS_SSE) as resp:
                    resp.raise_for_status()
                    for data_line in _iter_sse_data_lines(resp):
                        obj = json.loads(data_line)
                        result, err = _extract_result(obj)
                        if err is None:
                            return result
                        # If an error is embedded, raise immediately
                        raise RuntimeError(f"MCP error: {err}")
            except Exception:
                # Fall back to plain JSON
                pass

        # Plain JSON fallback
        resp = client.post(MCP_URL, json=payload, headers=HEADERS_JSON)
        resp.raise_for_status()
        obj = resp.json()
        result, err = _extract_result(obj)
        if err is not None:
            raise RuntimeError(f"MCP error: {err}")
        return result

def coerce_rolls(value: Any) -> list[int]:
    """
    Accepts:
      - list[int]
      - a JSON/text string like "[1,2,3]"
      - a comma/space separated string like "1,2,3" (last resort)
    Returns list[int] or raises ValueError.
    """
    if isinstance(value, list) and all(isinstance(x, int) for x in value):
        return value

    if isinstance(value, str):
        # Try to interpret as a Python/JSON list string
        try:
            parsed = ast.literal_eval(value)
            if isinstance(parsed, list) and all(isinstance(x, int) for x in parsed):
                return parsed
        except Exception:
            pass

        # Last resort: split by comma/space
        try:
            parts = [p for p in value.replace("[", "").replace("]", "").split(",")]
            nums = [int(p.strip()) for p in parts if p.strip() != ""]
            if nums:
                return nums
        except Exception:
            pass

    raise ValueError(f"Could not coerce rolls from: {value!r}")

def main():
    # 1) Roll dice
    rolled = call_tool("roll_dice", {"n_dice": 10}, stream_preferred=True)
    rolls = coerce_rolls(rolled)

    # 2) Sum dice
    sum_result = call_tool("sum_dice", {"rolls": rolls}, stream_preferred=True)

    print(f"roll_dice -> {rolls}")
    print(f"sum_dice  -> {sum_result}")

if __name__ == "__main__":
    main()


roll_dice -> [4, 5, 6, 2, 2, 6, 5, 5, 5, 3]
sum_dice  -> 43


In [65]:
print(sum([3, 4, 1, 1, 6, 1, 5, 3, 1, 4, 4, 4, 4, 1, 3, 3, 1, 2, 5, 5]))

61


In [2]:

import yfinance as yf
dat = yf.Ticker("MSFT")

dat

yfinance.Ticker object <MSFT>